In [3]:
import os 
os.chdir("C:/Downloads")
os.getcwd()

'C:\\Downloads'

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.modeling import models, fitting
from fitting_functions import *
import glob
from astropy.constants import c
from astropy.io import ascii
import astropy.units as u
from Borg_emission_lines import lines
import pickle
import random
import pprint
import os
import pickle
import numpy as np
import seaborn as sns

In [4]:
proc_df = pd.read_csv('processed_test_df.csv')
colnames = ['survey', 'field', 'prog_id', 'id', 'msaid', 'ra', 'dec', 'z', 
            'flag_lya', 'flag_agn', 'flag_z']

raw_df = pd.read_csv(
    'highz_msaid_public.dat',
    comment='#',
    delim_whitespace=True,
    names=colnames,
    header=None
)
raw_df.rename(columns={'msaid': 'Galaxy ID'}, inplace=True)


file_spec = 'spectra_13Dec2024_full.p'
with open(file_spec, 'rb') as file:
    data_spec = pickle.load(file)

tab = ascii.read('highz_msaid_public.dat')
ratio_lines = ['[OIII]_5007_flux_SNR', 'Hbeta_flux_SNR', '[NeIII]_flux_SNR', '[OII]_doublet_flux_SNR']

def get_spec_key(gal):
    PREFERRED_KEYS = [
        'prism-clear',
        'g140h-f100lp',
        'g140m-f100lp',
        'g140m-f070lp',
        'g235h-f170lp',
        'g235m-f170lp',
        'g395h-f290lp',
        'g395m-f290lp'
    ]
    keys = data_spec[gal].keys()
    for k in PREFERRED_KEYS:
        if k in keys:
            return k
    return None  

C:\Users\isaia\AppData\Local\Temp\ipykernel_33408\3371750413.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  raw_df = pd.read_csv(


In [5]:
glass_df = proc_df[proc_df['survey'] == 'glass']
borg_df = proc_df[proc_df['survey'] == 'borg']

glass_df



,survey,field,prog_id,id,Galaxy ID,ra,dec,z,flag_lya,flag_agn,...,NeIII_OII_ratio_err,OIII5007_Hbeta_ratio,OIII5007_Hbeta_ratio_err,Hdelta_Hbeta_ratio,Hdelta_Hbeta_ratio_err,Hgamma_Hbeta_ratio,Hgamma_Hbeta_ratio_err,MgI_5174_flux,MgI_5174_flux_err,MgI_5174_flux_SNR
473,glass,a2744,1324,10000,1324_10000,3.601339,-30.379225,7.891,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,glass,a2744,1324,100003,1324_100003,3.604509,-30.380444,7.880,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,glass,a2744,1324,100005,1324_100005,3.606458,-30.380994,7.883,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476,glass,a2744,1324,10003,1324_10003,3.617163,-30.425546,9.313,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
477,glass,a2744,1324,10021,1324_10021,3.608511,-30.418540,7.288,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,glass,a2744,1324,110000,1324_110000,3.570643,-30.414638,5.765,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
479,glass,a2744,1324,150008,1324_150008,3.602530,-30.419232,6.230,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,glass,a2744,1324,160122,1324_160122,3.564901,-30.424957,5.333,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,glass,a2744,1324,400009,1324_400009,3.600593,-30.410271,6.376,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,glass,a2744,1324,50002,1324_50002,3.577001,-30.415522,5.135,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Processing

In [7]:
def find_spectral_features(gal_ids, num_bootstraps):
    all_snr_values = []
    # Collect Base SNR in the data set. 
    all_fit_results = []
    for idx, gal in enumerate(gal_ids):
        try:
            spec_key = get_spec_key(gal)
            if spec_key is None:
                print(f"Skipping {gal}: no supported instrument found")
                continue
            itab = np.where(tab['msaid'] == gal)[0]
            z = tab['z'][itab][0]
            wavelength = data_spec[gal][spec_key]['lam']*1e4
            flux_nu = data_spec[gal][spec_key]['flux']
            flux_nu_err = data_spec[gal][spec_key]['err']

            flux_l = convert_fnu_to_flambda(wavelength, flux_nu) * 1.e20
            flux_l_err = convert_fnu_to_flambda(wavelength, flux_nu_err) * 1.e20
            


            # select indices in wavelength array that correspond to our selected wavelength range
            redshift_lines = {key: value * (z + 1) for key, value in lines.items()}
            spectral_windows = {
                'CIV_window': [redshift_lines['CIV_doublet'] - 1500, redshift_lines['CIII]_doublet'] + 1500],
                'OII_window': [redshift_lines['[OII]_doublet'] - 1500, redshift_lines['Hgammma_OIII'] + 1500],
                'Hb_OIII_window': [redshift_lines['Hbeta'] - 1500, redshift_lines['[OIII]_5007'] + 1500]
            }

            # Begin bootstrap sampling:
            N = num_bootstraps
            bootstrap_results = {
                'continuum_slope': [],
                'continuum_intercept': [],
                'line_fluxes': [],  # List of dictionaries, one per iteration
                'flux_ratios': [],  # List of dictionaries, one per iteration
                'amplitudes': [],   # Add amplitudes to the dictionary
                'widths': []        # Add widths to the dictionary
            }   

            
            # **Compute Errors from Bootstrap Samples**
            final_results = {}  # Reset for each galaxy!
            count=0
            for _ in range(N):  # loop over the noise and then get the windows, and at every iteration compute the line fluxes and ratios
                count+=1
                # Generate new spectrum by adding Gaussian noise
                flux_new = flux_l + np.random.normal(0, flux_l_err)

                # Dictionary to store this iteration's line fluxes
                iteration_fluxes = {}
                iteration_ratios = {}
                iteration_amplitudes = {}
                iteration_widths = {}

                for window_name, window_range in spectral_windows.items():
                    sliced_index = np.where((wavelength >= window_range[0]) & (wavelength <= window_range[1]))
                    window_wavelength = wavelength[sliced_index]
                    window_flux = flux_l[sliced_index]
                    window_flux_err = flux_l_err[sliced_index]
                    invalid_spec = np.where((np.isfinite(window_flux) == False) | (np.isfinite(window_flux_err) == False))
                    window_flux[invalid_spec] = 0.
                    window_flux_err[invalid_spec] = 1.e20
            
                    model = models.Polynomial1D(1)
                    Nlines = 0

                    for key in redshift_lines:
                        lam = redshift_lines[key]
                        if (lam < np.nanmin(window_wavelength)) | (lam > np.nanmax(window_wavelength)):
                            continue

                        line = models.Gaussian1D(amplitude=np.nanmax(window_flux[(window_wavelength > (lam - 100.)) & (window_wavelength < (lam + 100.))]),
                                                mean=lam, stddev=60)
                        line.amplitude.min = 0.  # 1.e-3
                        line.amplitude.max = np.nanmax(window_flux[(window_wavelength > (lam - 100.)) & (window_wavelength < (lam + 100.))])
                        line.amplitude.fixed = False
                        line.stddev.min = 0.  # 1.e-3
                        line.stddev.fixed = False
                        model += line
                        Nlines += 1

                    # Fit model:
                    fit_g = fitting.LevMarLSQFitter(calc_uncertainties=True)
                    g = fit_g(model, window_wavelength, flux_new[sliced_index], weights=1 / window_flux_err)
                    # Check fit status using fit_info
                    status = fit_g.fit_info.get('status')

                
                    # Store bootstrapped parameters for this iteration
                    bootstrap_results['continuum_slope'].append(g.parameters[0])  # Use g.parameters[0] for the slope (c0)
                    bootstrap_results['continuum_intercept'].append(g.parameters[1])  # Use g.parameters[1] for the intercept (c1)


                    #for line_name, lam in redshift_lines.items():
                    for i, line_name in enumerate([key for key in redshift_lines if redshift_lines[key] >= np.nanmin(window_wavelength) and redshift_lines[key] <= np.nanmax(window_wavelength)]):

                        #print(line_name)
                        amp = getattr(g, f'amplitude_{i+1}').value
                        width = getattr(g, f'stddev_{i+1}').value

                        # Store in dictionaries for later error calculation
                        if line_name not in iteration_amplitudes:
                            iteration_amplitudes[line_name] = []
                            iteration_widths[line_name] = []

                        iteration_amplitudes[line_name].append(amp)
                        iteration_widths[line_name].append(width)

                        # **Compute line flux**
                        flux = amp * width * np.sqrt(2 * np.pi) * 1.e-2
                        iteration_fluxes[f"{line_name}_flux"] = flux
                    # print(count) #correctly goes throush the 3 spectral windows and finds fluxes for all lines, this for 100 times


                # **Compute flux ratios**
                # print(iteration_fluxes) #Fluxes saved correctly! No overwritting
                if '[NeIII]_flux' in iteration_fluxes and '[OII]_doublet_flux' in iteration_fluxes:
                    iteration_ratios['NeIII_OII_ratio'] = iteration_fluxes['[NeIII]_flux'] / iteration_fluxes['[OII]_doublet_flux']

                if '[OIII]_5007_flux' in iteration_fluxes and 'Hbeta_flux' in iteration_fluxes:
                    iteration_ratios['OIII5007_Hbeta_ratio'] = iteration_fluxes['[OIII]_5007_flux'] / iteration_fluxes['Hbeta_flux']

                if 'Hdelta_flux' in iteration_fluxes and 'Hbeta_flux' in iteration_fluxes:
                    iteration_ratios['Hdelta_Hbeta_ratio'] = iteration_fluxes['Hdelta_flux'] / iteration_fluxes['Hbeta_flux']

                if 'Hgammma_OIII_flux' in iteration_fluxes and 'Hbeta_flux' in iteration_fluxes:
                    iteration_ratios['Hgamma_Hbeta_ratio'] = iteration_fluxes['Hgammma_OIII_flux'] / iteration_fluxes['Hbeta_flux']



                # Debugging: Print ratios for each iteration
                #print(f"Iteration {count} flux ratios: {iteration_ratios}") This worked!

                # Store this bootstrap iteration's results
                bootstrap_results['line_fluxes'].append(iteration_fluxes)
                bootstrap_results['flux_ratios'].append(iteration_ratios)
                bootstrap_results['amplitudes'].append(iteration_amplitudes)
                bootstrap_results['widths'].append(iteration_widths)

            # Now that all bootstrap iterations are done, plot and save the fit for each spectral window
            for window_name, window_range in spectral_windows.items():
                sliced_index = np.where((wavelength >= window_range[0]) & (wavelength <= window_range[1]))
                window_wavelength = wavelength[sliced_index]
                window_flux = flux_l[sliced_index]
                window_flux_err = flux_l_err[sliced_index]

                # Define the model and fit it
                model = models.Polynomial1D(1)
                Nlines = 0

                for key in redshift_lines:
                    lam = redshift_lines[key]
                    if (lam < np.nanmin(window_wavelength)) | (lam > np.nanmax(window_wavelength)):
                        continue

                    line = models.Gaussian1D(
                        amplitude=np.nanmax(window_flux[(window_wavelength > (lam - 100.)) & (window_wavelength < (lam + 100.))]),
                        mean=lam, stddev=60
                    )
                    line.amplitude.min = 0.
                    line.amplitude.max = np.nanmax(window_flux[(window_wavelength > (lam - 100.)) & (window_wavelength < (lam + 100.))])
                    model += line
                    Nlines += 1

                # Fit the model to the spectrum
                fit_g = fitting.LevMarLSQFitter(calc_uncertainties=True)
                g = fit_g(model, window_wavelength, window_flux, weights=1 / window_flux_err, maxiter=10000, estimate_jacobian=True)

                #############################################################################################################
                #Add Fit Diagnostics (Metric to see which galaxies fit poorly) 
                #Compute RMSE of residual, coming from subtracting observed flux in specific wavelength from the
                #fit function to the data in that same wavelength window 
                residuals = window_flux - g(window_wavelength)
                
                rms = np.sqrt(np.mean(residuals**2)) 
                print(f"RMS for {gal} in {window_name}: {rms:.4g}")
                final_results[f"{window_name}_fit_rms"] = rms
                
                
                #############################################################################################################
                # Plot and save the fit for each spectral window
                fig = plt.figure(figsize=(7, 7))
                plt.title(gal + " - " + window_name, fontsize=16)
                plt.step(window_wavelength, window_flux, where='mid', color='k', label='Data')  # Original data
                plt.step(window_wavelength, window_flux_err, where='mid', color='grey', linestyle='dotted', label='Error')  # Error bars
                residuals = window_flux - g(window_wavelength)  # Residuals
                plt.step(window_wavelength, residuals, where='mid', color='orange', label='Residuals')  # Residuals
                plt.plot(window_wavelength, g(window_wavelength), color='red', label='Fit')  # Model fit

                plt.xlabel(r"Wavelength ($\AA$)", fontsize=18)
                plt.ylabel(r"Flux (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)", fontsize=18)
                plt.xticks(fontsize=16)
                plt.yticks(fontsize=16)

                # Save the plot as a PNG (or any other format)
                plot_filename = f"Figures_spectra/fit_{gal}_{window_name}.png"
                plt.savefig(plot_filename)  # Save the figure with a unique filename based on the window name

                plt.close()  # Close the plot after saving

            # **Find all emission lines across bootstraps**
            all_detected_lines = set()
            for iteration in bootstrap_results['line_fluxes']:
                all_detected_lines.update(iteration.keys())  # Collect all line_flux keys

            # **Compute errors for all detected lines**
            for line in all_detected_lines:
                flux_values = [iteration[line] for iteration in bootstrap_results['line_fluxes'] if line in iteration]
                if len(flux_values) > 0:  # Ensure at least one detection
                    flux_16, flux_median, flux_84 = np.percentile(flux_values, [16, 50, 84])
                    final_results[line] = flux_median
                    final_results[f"{line}_err"] = np.array([flux_median - flux_16, flux_84 - flux_median])  # Asymmetric errors
                    err_minus = flux_median - flux_16
                    err_plus = flux_84 - flux_median 
                    avg_error = (err_minus + err_plus) / 2 #Gives STD. 
                    line_SNR = flux_median / avg_error if avg_error > 0 else 0 #Avoid divisions by 0
                    final_results[f"{line}_SNR"] = line_SNR #Store SNR Values
                    
            #SNR Checking       
        
            # **Compute errors for flux ratios**
            for ratio in ['NeIII_OII_ratio', 'OIII5007_Hbeta_ratio','Hdelta_Hbeta_ratio','Hgamma_Hbeta_ratio']:
                ratio_values = [iteration[ratio] for iteration in bootstrap_results['flux_ratios'] if ratio in iteration]
                if len(ratio_values) > 0:
                    ratio_16, ratio_median, ratio_84 = np.percentile(ratio_values, [16, 50, 84])
                    final_results[ratio] = ratio_median
                    final_results[f"{ratio}_err"] = np.array([ratio_median - ratio_16, ratio_84 - ratio_median])                      

            # Append results **per galaxy**
            all_fit_results.append({gal: final_results})
            print(f"Final results for {gal}:", final_results.keys())
        except Exception as e: 
            print(f"Failed on {gal} due to: {e}")
    print(f"Processed {len(all_fit_results)} galaxies")
    return(all_fit_results)

In [10]:
borg_results = find_spectral_features(list(borg_df['Galaxy ID']), 100)

c:\Users\isaia\anaconda3\Lib\site-packages\astropy\modeling\functional_models.py:218: RuntimeWarning: divide by zero encountered in divide
  return amplitude * np.exp(-0.5 * (x - mean) ** 2 / stddev**2)


RMS for 1747_1081 in CIV_window: 0.2823
RMS for 1747_1081 in OII_window: 0.1496
RMS for 1747_1081 in Hb_OIII_window: 0.1461
Final results for 1747_1081: dict_keys(['CIV_window_fit_rms', 'OII_window_fit_rms', 'Hb_OIII_window_fit_rms', '[NeIII]_flux', '[NeIII]_flux_err', '[NeIII]_flux_SNR', '[OIII]_5007_flux', '[OIII]_5007_flux_err', '[OIII]_5007_flux_SNR', 'Hdelta_flux', 'Hdelta_flux_err', 'Hdelta_flux_SNR', 'CIV_doublet_flux', 'CIV_doublet_flux_err', 'CIV_doublet_flux_SNR', '[OII]_doublet_flux', '[OII]_doublet_flux_err', '[OII]_doublet_flux_SNR', 'Hepsilon_flux', 'Hepsilon_flux_err', 'Hepsilon_flux_SNR', '[OIII]_4959_flux', '[OIII]_4959_flux_err', '[OIII]_4959_flux_SNR', 'OIII]_doublet_flux', 'OIII]_doublet_flux_err', 'OIII]_doublet_flux_SNR', 'Hgammma_OIII_flux', 'Hgammma_OIII_flux_err', 'Hgammma_OIII_flux_SNR', 'Hbeta_flux', 'Hbeta_flux_err', 'Hbeta_flux_SNR', 'CIII]_doublet_flux', 'CIII]_doublet_flux_err', 'CIII]_doublet_flux_SNR', 'NeIII_OII_ratio', 'NeIII_OII_ratio_err', 'OIII5007

C:\Users\isaia\AppData\Local\Temp\ipykernel_33408\4219292893.py:119: RuntimeWarning: divide by zero encountered in scalar divide
  iteration_ratios['NeIII_OII_ratio'] = iteration_fluxes['[NeIII]_flux'] / iteration_fluxes['[OII]_doublet_flux']


RMS for 1747_1084 in CIV_window: 0.3642
RMS for 1747_1084 in OII_window: 0.1231
RMS for 1747_1084 in Hb_OIII_window: 0.09897
Final results for 1747_1084: dict_keys(['CIV_window_fit_rms', 'OII_window_fit_rms', 'Hb_OIII_window_fit_rms', '[NeIII]_flux', '[NeIII]_flux_err', '[NeIII]_flux_SNR', '[OIII]_5007_flux', '[OIII]_5007_flux_err', '[OIII]_5007_flux_SNR', 'Hdelta_flux', 'Hdelta_flux_err', 'Hdelta_flux_SNR', 'CIV_doublet_flux', 'CIV_doublet_flux_err', 'CIV_doublet_flux_SNR', '[OII]_doublet_flux', '[OII]_doublet_flux_err', '[OII]_doublet_flux_SNR', 'Hepsilon_flux', 'Hepsilon_flux_err', 'Hepsilon_flux_SNR', '[OIII]_4959_flux', '[OIII]_4959_flux_err', '[OIII]_4959_flux_SNR', 'OIII]_doublet_flux', 'OIII]_doublet_flux_err', 'OIII]_doublet_flux_SNR', 'MgI_5174_flux', 'MgI_5174_flux_err', 'MgI_5174_flux_SNR', 'Hgammma_OIII_flux', 'Hgammma_OIII_flux_err', 'Hgammma_OIII_flux_SNR', 'Hbeta_flux', 'Hbeta_flux_err', 'Hbeta_flux_SNR', 'CIII]_doublet_flux', 'CIII]_doublet_flux_err', 'CIII]_doublet_fl

C:\Users\isaia\AppData\Local\Temp\ipykernel_33408\4219292893.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  iteration_ratios['OIII5007_Hbeta_ratio'] = iteration_fluxes['[OIII]_5007_flux'] / iteration_fluxes['Hbeta_flux']
C:\Users\isaia\AppData\Local\Temp\ipykernel_33408\4219292893.py:125: RuntimeWarning: divide by zero encountered in scalar divide
  iteration_ratios['Hdelta_Hbeta_ratio'] = iteration_fluxes['Hdelta_flux'] / iteration_fluxes['Hbeta_flux']
C:\Users\isaia\AppData\Local\Temp\ipykernel_33408\4219292893.py:128: RuntimeWarning: divide by zero encountered in scalar divide
  iteration_ratios['Hgamma_Hbeta_ratio'] = iteration_fluxes['Hgammma_OIII_flux'] / iteration_fluxes['Hbeta_flux']
C:\Users\isaia\AppData\Local\Temp\ipykernel_33408\4219292893.py:128: RuntimeWarning: invalid value encountered in scalar divide
  iteration_ratios['Hgamma_Hbeta_ratio'] = iteration_fluxes['Hgammma_OIII_flux'] / iteration_fluxes['Hbeta_flux']
C:\Users\isaia\AppData\Local\Temp\

RMS for 1747_1257 in CIV_window: 0.5153
RMS for 1747_1257 in OII_window: 0.1187
RMS for 1747_1257 in Hb_OIII_window: 0.0997
Final results for 1747_1257: dict_keys(['CIV_window_fit_rms', 'OII_window_fit_rms', 'Hb_OIII_window_fit_rms', '[NeIII]_flux', '[NeIII]_flux_err', '[NeIII]_flux_SNR', '[OIII]_5007_flux', '[OIII]_5007_flux_err', '[OIII]_5007_flux_SNR', 'Hdelta_flux', 'Hdelta_flux_err', 'Hdelta_flux_SNR', 'CIV_doublet_flux', 'CIV_doublet_flux_err', 'CIV_doublet_flux_SNR', '[OII]_doublet_flux', '[OII]_doublet_flux_err', '[OII]_doublet_flux_SNR', 'Hepsilon_flux', 'Hepsilon_flux_err', 'Hepsilon_flux_SNR', '[OIII]_4959_flux', '[OIII]_4959_flux_err', '[OIII]_4959_flux_SNR', 'OIII]_doublet_flux', 'OIII]_doublet_flux_err', 'OIII]_doublet_flux_SNR', 'MgI_5174_flux', 'MgI_5174_flux_err', 'MgI_5174_flux_SNR', 'Hgammma_OIII_flux', 'Hgammma_OIII_flux_err', 'Hgammma_OIII_flux_SNR', 'Hbeta_flux', 'Hbeta_flux_err', 'Hbeta_flux_SNR', 'CIII]_doublet_flux', 'CIII]_doublet_flux_err', 'CIII]_doublet_flu

c:\Users\isaia\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


RMS for 1747_138 in CIV_window: 0.3082
RMS for 1747_138 in OII_window: 0.1071
RMS for 1747_138 in Hb_OIII_window: 0.1447
Final results for 1747_138: dict_keys(['CIV_window_fit_rms', 'OII_window_fit_rms', 'Hb_OIII_window_fit_rms', '[NeIII]_flux', '[NeIII]_flux_err', '[NeIII]_flux_SNR', '[OIII]_5007_flux', '[OIII]_5007_flux_err', '[OIII]_5007_flux_SNR', 'Hdelta_flux', 'Hdelta_flux_err', 'Hdelta_flux_SNR', 'CIV_doublet_flux', 'CIV_doublet_flux_err', 'CIV_doublet_flux_SNR', '[OII]_doublet_flux', '[OII]_doublet_flux_err', '[OII]_doublet_flux_SNR', 'Hepsilon_flux', 'Hepsilon_flux_err', 'Hepsilon_flux_SNR', '[OIII]_4959_flux', '[OIII]_4959_flux_err', '[OIII]_4959_flux_SNR', 'OIII]_doublet_flux', 'OIII]_doublet_flux_err', 'OIII]_doublet_flux_SNR', 'MgI_5174_flux', 'MgI_5174_flux_err', 'MgI_5174_flux_SNR', 'Hgammma_OIII_flux', 'Hgammma_OIII_flux_err', 'Hgammma_OIII_flux_SNR', 'Hbeta_flux', 'Hbeta_flux_err', 'Hbeta_flux_SNR', 'CIII]_doublet_flux', 'CIII]_doublet_flux_err', 'CIII]_doublet_flux_SN

## Try Bins

In [8]:
glass_bin_7 = glass_df[(glass_df['z'] >= 6.5) & (glass_df['z'] >= 7.5)]

In [11]:
import pandas as pd

rows = []

# Iterate over the list
for item in borg_results:
    # Each `item` is a dict with one key-value pair
    for gal_id, data in item.items():
        row = {'Galaxy ID': gal_id}
        for key, value in data.items():
            row[key] = value
        rows.append(row)

borg_df = pd.DataFrame(rows)

In [12]:
glass_7_results = find_spectral_features(list(glass_bin_7['Galaxy ID']), 100)

Failed on 1324_10000 due to: Objective function has encountered a non-finite value, this will cause the fit to fail!
Please remove non-finite values from your input data before fitting to avoid this error.
Failed on 1324_100003 due to: Objective function has encountered a non-finite value, this will cause the fit to fail!
Please remove non-finite values from your input data before fitting to avoid this error.
Failed on 1324_100005 due to: Objective function has encountered a non-finite value, this will cause the fit to fail!
Please remove non-finite values from your input data before fitting to avoid this error.
Failed on 1324_10003 due to: Objective function has encountered a non-finite value, this will cause the fit to fail!
Please remove non-finite values from your input data before fitting to avoid this error.
Processed 0 galaxies


## Are the **ID's** in data_spec? 

In [14]:
glass_ids = list(glass_df['Galaxy ID'])

for key in set(data_spec.keys()): 
    if key in(glass_ids):
        print(f"Found {key} in data_spec")

Found 1324_100003 in data_spec
Found 1324_70018 in data_spec
Found 1324_10003 in data_spec
Found 1324_160122 in data_spec
Found 1324_100005 in data_spec
Found 1324_400009 in data_spec
Found 1324_50002 in data_spec
Found 1324_10000 in data_spec
Found 1324_50038 in data_spec
Found 1324_110000 in data_spec
Found 1324_150008 in data_spec
Found 1324_10021 in data_spec


## What percentage of **flux** values in each Glass galaxy are non-finite? 

In [15]:
for gid in glass_ids:
    if gid not in data_spec:
        print(f"{gid} is missing from data_spec")
        continue

    for inst in data_spec[gid]:
        spec = data_spec[gid][inst]
        lam = spec['lam']
        flux = spec['flux']
        err = spec['err']

        # Safety: convert to NumPy arrays
        flux = np.array(flux)
        err = np.array(err)

        # Check non-finite values in flux
        total_flux_pts = len(flux)
        nonfinite_mask = ~np.isfinite(flux)
        num_nonfinite = np.sum(nonfinite_mask)

        if total_flux_pts == 0:
            print(f"{gid} ({inst}): EMPTY flux array")
        elif num_nonfinite == total_flux_pts:
            print(f"{gid} ({inst}): ❌ All {total_flux_pts} flux values are non-finite")
        elif num_nonfinite > 0:
            percent = (num_nonfinite / total_flux_pts) * 100
            print(f"{gid} ({inst}): ⚠️ {percent:.2f}% of flux values are non-finite")
        else:
            print(f"{gid} ({inst}): ✅ All flux values are finite")


1324_10000 (g140h-f100lp): ⚠️ 49.06% of flux values are non-finite
1324_10000 (g235h-f170lp): ⚠️ 34.74% of flux values are non-finite
1324_10000 (g395h-f290lp): ⚠️ 31.53% of flux values are non-finite
1324_100003 (g140h-f100lp): ⚠️ 51.05% of flux values are non-finite
1324_100003 (g235h-f170lp): ⚠️ 37.44% of flux values are non-finite
1324_100003 (g395h-f290lp): ⚠️ 34.39% of flux values are non-finite
1324_100005 (g140h-f100lp): ⚠️ 52.26% of flux values are non-finite
1324_100005 (g235h-f170lp): ⚠️ 39.09% of flux values are non-finite
1324_100005 (g395h-f290lp): ⚠️ 36.11% of flux values are non-finite
1324_10003 (g140h-f100lp): ⚠️ 60.88% of flux values are non-finite
1324_10003 (g235h-f170lp): ⚠️ 48.13% of flux values are non-finite
1324_10003 (g395h-f290lp): ⚠️ 45.66% of flux values are non-finite
1324_10021 (g140h-f100lp): ⚠️ 56.40% of flux values are non-finite
1324_10021 (g235h-f170lp): ⚠️ 44.71% of flux values are non-finite
1324_10021 (g395h-f290lp): ⚠️ 42.00% of flux values are 

## What percentage of **error** values in this each Glass galaxy are non-finite? 

In [16]:
for gid in glass_ids:
    if gid not in data_spec:
        print(f"{gid} is missing from data_spec")
        continue

    for inst in data_spec[gid]:
        spec = data_spec[gid][inst]
        lam = spec['lam']
        err = spec['err']

        # Safety: convert to NumPy array
        err = np.array(err)

        total_pts = len(err)
        nonfinite_mask = ~np.isfinite(err)
        num_nonfinite = np.sum(nonfinite_mask)

        if total_pts == 0:
            print(f"{gid} ({inst}): ❌ EMPTY error array")
        elif num_nonfinite == total_pts:
            print(f"{gid} ({inst}): ❌ All {total_pts} error values are non-finite")
        elif num_nonfinite > 0:
            percent = (num_nonfinite / total_pts) * 100
            print(f"{gid} ({inst}): ⚠️ {percent:.2f}% of error values are non-finite")
        else:
            print(f"{gid} ({inst}): ✅ All error values are finite")


1324_10000 (g140h-f100lp): ⚠️ 48.91% of error values are non-finite
1324_10000 (g235h-f170lp): ⚠️ 34.54% of error values are non-finite
1324_10000 (g395h-f290lp): ⚠️ 31.32% of error values are non-finite
1324_100003 (g140h-f100lp): ⚠️ 50.90% of error values are non-finite
1324_100003 (g235h-f170lp): ⚠️ 37.24% of error values are non-finite
1324_100003 (g395h-f290lp): ⚠️ 34.18% of error values are non-finite
1324_100005 (g140h-f100lp): ⚠️ 52.11% of error values are non-finite
1324_100005 (g235h-f170lp): ⚠️ 38.89% of error values are non-finite
1324_100005 (g395h-f290lp): ⚠️ 35.89% of error values are non-finite
1324_10003 (g140h-f100lp): ⚠️ 60.80% of error values are non-finite
1324_10003 (g235h-f170lp): ⚠️ 48.03% of error values are non-finite
1324_10003 (g395h-f290lp): ⚠️ 45.55% of error values are non-finite
1324_10021 (g140h-f100lp): ⚠️ 56.25% of error values are non-finite
1324_10021 (g235h-f170lp): ⚠️ 44.51% of error values are non-finite
1324_10021 (g395h-f290lp): ⚠️ 41.79% of er

## What percentage of  values in this each Glass galaxy are **zero**? 

In [17]:
for gid in glass_ids:
    if gid not in data_spec:
        print(f"{gid} is missing from data_spec")
        continue

    for inst in data_spec[gid]:
        spec = data_spec[gid][inst]
        err = np.array(spec['err'])

        total_pts = len(err)
        num_zero_err = np.sum(err == 0)
        percent_zero_err = (num_zero_err / total_pts) * 100 if total_pts > 0 else 0

        if num_zero_err == total_pts:
            print(f"{gid} ({inst}): ❌ All {total_pts} error values are zero")
        elif num_zero_err > 0:
            print(f"{gid} ({inst}): ⚠️ {num_zero_err} error values are zero ({percent_zero_err:.2f}%)")
        else:
            print(f"{gid} ({inst}): ✅ No zero error values")


1324_10000 (g140h-f100lp): ✅ No zero error values
1324_10000 (g235h-f170lp): ✅ No zero error values
1324_10000 (g395h-f290lp): ✅ No zero error values
1324_100003 (g140h-f100lp): ✅ No zero error values
1324_100003 (g235h-f170lp): ✅ No zero error values
1324_100003 (g395h-f290lp): ✅ No zero error values
1324_100005 (g140h-f100lp): ✅ No zero error values
1324_100005 (g235h-f170lp): ✅ No zero error values
1324_100005 (g395h-f290lp): ✅ No zero error values
1324_10003 (g140h-f100lp): ✅ No zero error values
1324_10003 (g235h-f170lp): ✅ No zero error values
1324_10003 (g395h-f290lp): ✅ No zero error values
1324_10021 (g140h-f100lp): ✅ No zero error values
1324_10021 (g235h-f170lp): ✅ No zero error values
1324_10021 (g395h-f290lp): ✅ No zero error values
1324_110000 (g140h-f100lp): ✅ No zero error values
1324_110000 (g235h-f170lp): ✅ No zero error values
1324_110000 (g395h-f290lp): ✅ No zero error values
1324_150008 (g140h-f100lp): ✅ No zero error values
1324_150008 (g235h-f170lp): ✅ No zero er

In [18]:
find_spectral_features([gid.strip() for gid in ['1324_70018', '1324_70018 ']], 100)

Failed on 1324_70018 due to: Objective function has encountered a non-finite value, this will cause the fit to fail!
Please remove non-finite values from your input data before fitting to avoid this error.
Failed on 1324_70018 due to: Objective function has encountered a non-finite value, this will cause the fit to fail!
Please remove non-finite values from your input data before fitting to avoid this error.
Processed 0 galaxies


C:\Users\isaia\AppData\Local\Temp\ipykernel_33408\4219292893.py:85: RuntimeWarning: divide by zero encountered in divide
  g = fit_g(model, window_wavelength, flux_new[sliced_index], weights=1 / window_flux_err)
c:\Users\isaia\anaconda3\Lib\site-packages\astropy\modeling\fitting.py:1139: RuntimeWarning: invalid value encountered in multiply
  value = np.ravel(weights * (model(*args[2:-1]) - meas))


[]

## Filtering out non-finite and 0 values (Test)

In [19]:
def find_spectral_features2(gal_ids, num_bootstraps):
    all_snr_values = []
    # Collect Base SNR in the data set. 
    all_fit_results = []
    for idx, gal in enumerate(gal_ids):
        try:
            spec_key = get_spec_key(gal)
            if spec_key is None:
                print(f"Skipping {gal}: no supported instrument found")
                continue
            itab = np.where(tab['msaid'] == gal)[0]
            z = tab['z'][itab][0]
            wavelength = data_spec[gal][spec_key]['lam']*1e4
            flux_nu = data_spec[gal][spec_key]['flux']
            flux_nu_err = data_spec[gal][spec_key]['err']

            flux_l = convert_fnu_to_flambda(wavelength, flux_nu) * 1.e20
            flux_l_err = convert_fnu_to_flambda(wavelength, flux_nu_err) * 1.e20
            


            # select indices in wavelength array that correspond to our selected wavelength range
            redshift_lines = {key: value * (z + 1) for key, value in lines.items()}
            spectral_windows = {
                'CIV_window': [redshift_lines['CIV_doublet'] - 1500, redshift_lines['CIII]_doublet'] + 1500],
                'OII_window': [redshift_lines['[OII]_doublet'] - 1500, redshift_lines['Hgammma_OIII'] + 1500],
                'Hb_OIII_window': [redshift_lines['Hbeta'] - 1500, redshift_lines['[OIII]_5007'] + 1500]
            }

            # Begin bootstrap sampling:
            N = num_bootstraps
            bootstrap_results = {
                'continuum_slope': [],
                'continuum_intercept': [],
                'line_fluxes': [],  # List of dictionaries, one per iteration
                'flux_ratios': [],  # List of dictionaries, one per iteration
                'amplitudes': [],   # Add amplitudes to the dictionary
                'widths': []        # Add widths to the dictionary
            }   

            
            # **Compute Errors from Bootstrap Samples**
            final_results = {}  # Reset for each galaxy!
            count=0
            for _ in range(N):  # loop over the noise and then get the windows, and at every iteration compute the line fluxes and ratios
                count+=1
                # Generate new spectrum by adding Gaussian noise
                flux_new = flux_l + np.random.normal(0, flux_l_err)

                # Dictionary to store this iteration's line fluxes
                iteration_fluxes = {}
                iteration_ratios = {}
                iteration_amplitudes = {}
                iteration_widths = {}

                for window_name, window_range in spectral_windows.items():
                    sliced_index = np.where((wavelength >= window_range[0]) & (wavelength <= window_range[1]))
                    window_wavelength = wavelength[sliced_index]
                    window_flux = flux_l[sliced_index]
                    window_flux_err = flux_l_err[sliced_index]

                    if len(window_wavelength) == 0 or np.all(~np.isfinite(window_flux)) or np.all(window_flux == 0):
                        print(f"Skipping plot fit for {gal} in {window_name}: empty or invalid data.")
                        continue

                    model = models.Polynomial1D(1)

                    Nlines = 0

                    for key in redshift_lines:
                        lam = redshift_lines[key]
                        if (lam < np.nanmin(window_wavelength)) | (lam > np.nanmax(window_wavelength)):
                            continue

                        line = models.Gaussian1D(amplitude=np.nanmax(window_flux[(window_wavelength > lam - 100) & (window_wavelength < lam + 100)], initial=0),
                                                mean=lam, stddev=60)
                        line.amplitude.min = 0.  # 1.e-3
                        line.amplitude.max = np.nanmax(window_flux[(window_wavelength > lam - 100) & (window_wavelength < lam + 100)], initial=0)
                        line.amplitude.fixed = False
                        line.stddev.min = 0.  # 1.e-3
                        line.stddev.fixed = False
                        model += line
                        Nlines += 1

                    # Fit model:
                    fit_g = fitting.LevMarLSQFitter(calc_uncertainties=True)
                    # Remove non-finite entries
                    valid_mask = (
                        np.isfinite(window_wavelength) &
                        np.isfinite(flux_new[sliced_index]) &
                        np.isfinite(window_flux_err) &
                        (window_flux_err > 0)
                    )

                    if np.count_nonzero(valid_mask) < 5:
                        print(f"Skipping fit for {gal} in {window_name}: not enough valid points.")
                        continue  # Skip this window entirely

                    x_fit = window_wavelength[valid_mask]
                    y_fit = flux_new[sliced_index][valid_mask]
                    w_fit = 1 / window_flux_err[valid_mask]

                    try:
                        g = fit_g(model, x_fit, y_fit, weights=w_fit)
                    except Exception as e:
                        print(f"Fit failed for {gal} in {window_name}: {e}")
                        continue

                    # Check fit status using fit_info
                    status = fit_g.fit_info.get('status')

                
                    # Store bootstrapped parameters for this iteration
                    bootstrap_results['continuum_slope'].append(g.parameters[0])  # Use g.parameters[0] for the slope (c0)
                    bootstrap_results['continuum_intercept'].append(g.parameters[1])  # Use g.parameters[1] for the intercept (c1)


                    #for line_name, lam in redshift_lines.items():
                    for i, line_name in enumerate([key for key in redshift_lines if redshift_lines[key] >= np.nanmin(window_wavelength) and redshift_lines[key] <= np.nanmax(window_wavelength)]):

                        #print(line_name)
                        amp = getattr(g, f'amplitude_{i+1}').value
                        width = getattr(g, f'stddev_{i+1}').value

                        # Store in dictionaries for later error calculation
                        if line_name not in iteration_amplitudes:
                            iteration_amplitudes[line_name] = []
                            iteration_widths[line_name] = []

                        iteration_amplitudes[line_name].append(amp)
                        iteration_widths[line_name].append(width)

                        # **Compute line flux**
                        flux = amp * width * np.sqrt(2 * np.pi) * 1.e-2
                        iteration_fluxes[f"{line_name}_flux"] = flux
                    # print(count) #correctly goes throush the 3 spectral windows and finds fluxes for all lines, this for 100 times


                # **Compute flux ratios**
                # print(iteration_fluxes) #Fluxes saved correctly! No overwritting
                if '[NeIII]_flux' in iteration_fluxes and '[OII]_doublet_flux' in iteration_fluxes:
                    iteration_ratios['NeIII_OII_ratio'] = iteration_fluxes['[NeIII]_flux'] / iteration_fluxes['[OII]_doublet_flux']

                if '[OIII]_5007_flux' in iteration_fluxes and 'Hbeta_flux' in iteration_fluxes:
                    iteration_ratios['OIII5007_Hbeta_ratio'] = iteration_fluxes['[OIII]_5007_flux'] / iteration_fluxes['Hbeta_flux']

                if 'Hdelta_flux' in iteration_fluxes and 'Hbeta_flux' in iteration_fluxes:
                    iteration_ratios['Hdelta_Hbeta_ratio'] = iteration_fluxes['Hdelta_flux'] / iteration_fluxes['Hbeta_flux']

                if 'Hgammma_OIII_flux' in iteration_fluxes and 'Hbeta_flux' in iteration_fluxes:
                    iteration_ratios['Hgamma_Hbeta_ratio'] = iteration_fluxes['Hgammma_OIII_flux'] / iteration_fluxes['Hbeta_flux']



                # Debugging: Print ratios for each iteration
                #print(f"Iteration {count} flux ratios: {iteration_ratios}") This worked!

                # Store this bootstrap iteration's results
                bootstrap_results['line_fluxes'].append(iteration_fluxes)
                bootstrap_results['flux_ratios'].append(iteration_ratios)
                bootstrap_results['amplitudes'].append(iteration_amplitudes)
                bootstrap_results['widths'].append(iteration_widths)

            # Now that all bootstrap iterations are done, plot and save the fit for each spectral window
            for window_name, window_range in spectral_windows.items():
                sliced_index = np.where((wavelength >= window_range[0]) & (wavelength <= window_range[1]))
                window_wavelength = wavelength[sliced_index]
                window_flux = flux_l[sliced_index]
                window_flux_err = flux_l_err[sliced_index]

                # Now safely check
                if len(window_wavelength) == 0 or np.all(~np.isfinite(window_flux)) or np.all(window_flux == 0):
                    print(f"Skipping window {window_name} for {gal}: invalid or empty flux data.")
                    continue

                if np.all(np.isnan(window_flux)) or np.all(window_flux == 0):
                    print(f"Skipping {gal} in {window_name}: flux is all NaNs or zero.")
                    continue


                # Define the model and fit it
                model = models.Polynomial1D(1)
                Nlines = 0

                for key in redshift_lines:
                    lam = redshift_lines[key]
                    if (lam < np.nanmin(window_wavelength)) | (lam > np.nanmax(window_wavelength)):
                        continue

                    line = models.Gaussian1D(
                        amplitude=np.nanmax(window_flux[(window_wavelength > lam - 100) & (window_wavelength < lam + 100)], initial=0),
                        mean=lam, stddev=60
                    )
                    line.amplitude.min = 0.
                    line.amplitude.max = np.nanmax(window_flux[(window_wavelength > lam - 100) & (window_wavelength < lam + 100)], initial=0)
                    model += line
                    Nlines += 1

                # Fit the model to the spectrum
                # Clean input arrays
                valid_mask = (
                    np.isfinite(window_wavelength) &
                    np.isfinite(window_flux) &
                    np.isfinite(window_flux_err) &
                    (window_flux_err > 0)
                )

                if np.count_nonzero(valid_mask) < 5:
                    print(f"Skipping final fit for {gal} in {window_name}: not enough valid data points.")
                    continue

                x_fit = window_wavelength[valid_mask]
                y_fit = window_flux[valid_mask]
                w_fit = 1 / window_flux_err[valid_mask]

                fit_g = fitting.LevMarLSQFitter(calc_uncertainties=True)

                try:
                    g = fit_g(model, x_fit, y_fit, weights=w_fit, maxiter=10000, estimate_jacobian=True)
                except Exception as e:
                    print(f"Final fit failed for {gal} in {window_name}: {e}")
                    continue


                #############################################################################################################
                #Add Fit Diagnostics (Metric to see which galaxies fit poorly) 
                #Compute RMSE of residual, coming from subtracting observed flux in specific wavelength from the
                #fit function to the data in that same wavelength window 
                residuals = window_flux - g(window_wavelength)
                
                rms = np.sqrt(np.mean(residuals**2)) 
                print(f"RMS for {gal} in {window_name}: {rms:.4g}")
                final_results[f"{window_name}_fit_rms"] = rms
                
                
                #############################################################################################################
                # Plot and save the fit for each spectral window
                fig = plt.figure(figsize=(7, 7))
                plt.title(gal + " - " + window_name, fontsize=16)
                plt.step(window_wavelength, window_flux, where='mid', color='k', label='Data')  # Original data
                plt.step(window_wavelength, window_flux_err, where='mid', color='grey', linestyle='dotted', label='Error')  # Error bars
                residuals = window_flux - g(window_wavelength)  # Residuals
                plt.step(window_wavelength, residuals, where='mid', color='orange', label='Residuals')  # Residuals
                plt.plot(window_wavelength, g(window_wavelength), color='red', label='Fit')  # Model fit

                plt.xlabel(r"Wavelength ($\AA$)", fontsize=18)
                plt.ylabel(r"Flux (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)", fontsize=18)
                plt.xticks(fontsize=16)
                plt.yticks(fontsize=16)

                # Save the plot as a PNG (or any other format)
                plot_filename = f"Figures_spectra/fit_{gal}_{window_name}.png"
                plt.savefig(plot_filename)  # Save the figure with a unique filename based on the window name

                plt.close()  # Close the plot after saving

            # **Find all emission lines across bootstraps**
            all_detected_lines = set()
            for iteration in bootstrap_results['line_fluxes']:
                all_detected_lines.update(iteration.keys())  # Collect all line_flux keys

            # **Compute errors for all detected lines**
            for line in all_detected_lines:
                flux_values = [iteration[line] for iteration in bootstrap_results['line_fluxes'] if line in iteration]
                if len(flux_values) > 0:  # Ensure at least one detection
                    flux_16, flux_median, flux_84 = np.percentile(flux_values, [16, 50, 84])
                    final_results[line] = flux_median
                    final_results[f"{line}_err"] = np.array([flux_median - flux_16, flux_84 - flux_median])  # Asymmetric errors
                    err_minus = flux_median - flux_16
                    err_plus = flux_84 - flux_median 
                    avg_error = (err_minus + err_plus) / 2 #Gives STD. 
                    line_SNR = flux_median / avg_error if avg_error > 0 else 0 #Avoid divisions by 0
                    final_results[f"{line}_SNR"] = line_SNR #Store SNR Values
                    
            #SNR Checking       
        
            # **Compute errors for flux ratios**
            for ratio in ['NeIII_OII_ratio', 'OIII5007_Hbeta_ratio','Hdelta_Hbeta_ratio','Hgamma_Hbeta_ratio']:
                ratio_values = [iteration[ratio] for iteration in bootstrap_results['flux_ratios'] if ratio in iteration]
                if len(ratio_values) > 0:
                    ratio_16, ratio_median, ratio_84 = np.percentile(ratio_values, [16, 50, 84])
                    final_results[ratio] = ratio_median
                    final_results[f"{ratio}_err"] = np.array([ratio_median - ratio_16, ratio_84 - ratio_median])
                else:
                    final_results[ratio] = np.nan
                    final_results[f"{ratio}_err"] = np.array([np.nan, np.nan])
                      

            # Append results **per galaxy**
            all_fit_results.append({gal: final_results})
            print(f"Final results for {gal}:", final_results.keys())
        except Exception as e: 
            print(f"Failed on {gal} due to: {e}")
    print(f"Processed {len(all_fit_results)} galaxies")
    return(all_fit_results)

In [20]:
glass_results = find_spectral_features2(glass_ids, 100)

Skipping plot fit for 1324_10000 in OII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in Hb_OIII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in OII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in Hb_OIII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in OII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in Hb_OIII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in OII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in Hb_OIII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in OII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in Hb_OIII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in OII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in Hb_OIII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in OII_window: empty or invalid data.
Skipping plot fit for 1324_10000 in Hb_OIII_window: e

In [21]:
glass_results[0]

{'1324_10000': {'CIV_window_fit_rms': nan,
  'CIV_doublet_flux': 0.0,
  'CIV_doublet_flux_err': array([0.        , 0.37920537]),
  'CIV_doublet_flux_SNR': 0.0,
  'CIII]_doublet_flux': 0.0,
  'CIII]_doublet_flux_err': array([0., 0.]),
  'CIII]_doublet_flux_SNR': 0,
  'OIII]_doublet_flux': 0.2080282256973873,
  'OIII]_doublet_flux_err': array([0.20802823, 0.41472213]),
  'OIII]_doublet_flux_SNR': 0.6680950804008359,
  'NeIII_OII_ratio': nan,
  'NeIII_OII_ratio_err': array([nan, nan]),
  'OIII5007_Hbeta_ratio': nan,
  'OIII5007_Hbeta_ratio_err': array([nan, nan]),
  'Hdelta_Hbeta_ratio': nan,
  'Hdelta_Hbeta_ratio_err': array([nan, nan]),
  'Hgamma_Hbeta_ratio': nan,
  'Hgamma_Hbeta_ratio_err': array([nan, nan])}}

In [22]:
import pandas as pd

rows = []

# Iterate over the list
for item in glass_results:
    # Each `item` is a dict with one key-value pair
    for gal_id, data in item.items():
        row = {'Galaxy ID': gal_id}
        for key, value in data.items():
            row[key] = value
        rows.append(row)

proc_glass_df = pd.DataFrame(rows)

In [23]:
proc_glass_df

,Galaxy ID,CIV_window_fit_rms,CIV_doublet_flux,CIV_doublet_flux_err,CIV_doublet_flux_SNR,CIII]_doublet_flux,CIII]_doublet_flux_err,CIII]_doublet_flux_SNR,OIII]_doublet_flux,OIII]_doublet_flux_err,OIII]_doublet_flux_SNR,NeIII_OII_ratio,NeIII_OII_ratio_err,OIII5007_Hbeta_ratio,OIII5007_Hbeta_ratio_err,Hdelta_Hbeta_ratio,Hdelta_Hbeta_ratio_err,Hgamma_Hbeta_ratio,Hgamma_Hbeta_ratio_err
0,1324_10000,NaN,0.000000,"[0.0, 0.3792053717344278]",0.000000,0.000000,"[0.0, 0.0]",0.000000,0.208028,"[0.2080282256973873, 0.41472213364540844]",0.668095,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
1,1324_100003,NaN,0.241214,"[0.23828343197799015, 0.7389186179175791]",0.493682,0.000000,"[0.0, 0.0]",0.000000,0.094590,"[0.0945899511252315, 0.3719344165932581]",0.405509,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
2,1324_100005,NaN,0.146888,"[0.14688776818164814, 0.29174336690251335]",0.669755,0.000000,"[0.0, 0.0]",0.000000,0.321122,"[0.32112217702623225, 0.5561890375850096]",0.732060,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
3,1324_10003,NaN,0.000000,"[0.0, 0.0]",0.000000,NaN,NaN,NaN,0.000000,"[0.0, 0.0]",0.000000,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
4,1324_10021,NaN,0.293569,"[0.2935693082552902, 0.5004114741128742]",0.739487,0.000000,"[0.0, 0.0]",0.000000,0.724620,"[0.7246203777020883, 1.7958532458719307]",0.574987,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
5,1324_110000,NaN,0.000000,"[0.0, 0.0]",0.000000,0.305087,"[0.28034031203247545, 0.31740140941587025]",1.020799,0.101357,"[0.1013572983323649, 0.1833170364311671]",0.712093,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
6,1324_150008,NaN,0.002731,"[0.0027309328367173496, 0.3243711153301399]",0.016698,1.055449,"[0.6643502074326805, 0.7897226071684185]",1.451714,0.787405,"[0.5340601017911548, 0.43467632733193784]",1.625632,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
7,1324_160122,NaN,89.119701,"[89.11970055060408, 137.75323597089897]",0.785635,1.199503,"[0.538918097023967, 1.8922761579919232]",0.986760,0.334118,"[0.33411821069755554, 0.2536818249460985]",1.136843,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
8,1324_400009,NaN,0.000000,"[0.0, 0.21393120555618073]",0.000000,0.000000,"[0.0, 0.06196892055783426]",0.000000,0.082820,"[0.08282034427313319, 0.2685321098917398]",0.471437,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"
9,1324_50002,NaN,0.000000,"[0.0, 0.0]",0.000000,0.842699,"[0.519877812682439, 1.312851811767308]",0.919611,0.000000,"[0.0, 0.0]",0.000000,NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]",NaN,"[nan, nan]"


In [46]:
merged_glass_df = pd.merge(proc_glass_df, raw_df, on='Galaxy ID', how = 'outer')

In [47]:
merged_glass_df = merged_glass_df[merged_glass_df['survey'] == 'glass']

In [48]:
merged_glass_df

,Galaxy ID,CIV_window_fit_rms,CIII]_doublet_flux,CIII]_doublet_flux_err,CIII]_doublet_flux_SNR,CIV_doublet_flux,CIV_doublet_flux_err,CIV_doublet_flux_SNR,OIII]_doublet_flux,OIII]_doublet_flux_err,...,survey,field,prog_id,id,ra,dec,z,flag_lya,flag_agn,flag_z
473,1324_10000,NaN,0.000000,"[0.0, 0.0]",0.000000,0.022194,"[0.022194479326390012, 0.5077333281752245]",0.083764,0.241356,"[0.24135592238829756, 0.45983210582461026]",...,glass,a2744,1324,10000,3.601339,-30.379225,7.891,0,0,-99
474,1324_100003,NaN,0.000000,"[0.0, 0.0]",0.000000,0.357267,"[0.35117191393706115, 0.6131278558015487]",0.740987,0.012236,"[0.012236083237663668, 0.20556971336357965]",...,glass,a2744,1324,100003,3.604509,-30.380444,7.880,0,0,-99
475,1324_100005,NaN,0.000000,"[0.0, 0.0]",0.000000,0.197827,"[0.1978270755600334, 0.29441677515300435]",0.803777,0.203401,"[0.2034005466878861, 0.8086768564988434]",...,glass,a2744,1324,100005,3.606458,-30.380994,7.883,0,0,-99
476,1324_10003,NaN,NaN,NaN,NaN,0.000000,"[0.0, 0.0]",0.000000,0.000000,"[0.0, 0.0]",...,glass,a2744,1324,10003,3.617163,-30.425546,9.313,0,0,-99
477,1324_10021,NaN,0.000000,"[0.0, 0.0]",0.000000,0.333135,"[0.33313486415149096, 0.41417928137688154]",0.891552,0.464286,"[0.46428634230201893, 1.2545622352505992]",...,glass,a2744,1324,10021,3.608511,-30.418540,7.288,0,0,-99
478,1324_110000,NaN,0.333234,"[0.29706767532830824, 0.2699040042134332]",1.175487,0.000000,"[0.0, 0.008049169321885555]",0.000000,0.155360,"[0.1553597431665282, 0.20778023856857022]",...,glass,a2744,1324,110000,3.570643,-30.414638,5.765,0,0,-99
479,1324_150008,NaN,0.914036,"[0.5497908898958672, 0.8225215327920723]",1.332111,0.000000,"[0.0, 0.3307865481666497]",0.000000,0.897433,"[0.7037839537851529, 0.3386609448996377]",...,glass,a2744,1324,150008,3.602530,-30.419232,6.230,0,0,-99
480,1324_160122,NaN,1.599805,"[0.8330119144296132, 1.5450861714469855]",1.345449,36.755359,"[36.75535949977759, 184.0487299773702]",0.332923,0.355350,"[0.3553501352052236, 0.3935942507544262]",...,glass,a2744,1324,160122,3.564901,-30.424957,5.333,0,0,-99
481,1324_400009,NaN,0.000000,"[0.0, 0.15271311644948185]",0.000000,0.000000,"[0.0, 0.21375944780028577]",0.000000,0.082586,"[0.08258569266524951, 0.2229044820384653]",...,glass,a2744,1324,400009,3.600593,-30.410271,6.376,0,0,-99
482,1324_50002,NaN,0.922771,"[0.5440579653718205, 0.8762926637323247]",1.299357,0.000000,"[0.0, 0.0]",0.000000,0.000000,"[0.0, 0.0]",...,glass,a2744,1324,50002,3.577001,-30.415522,5.135,0,0,-99


In [49]:
merged_glass_df.to_csv('Glass DF (Testing)', index = False)

### Testing Plots on the "Glass DF" 

In [50]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import ast

def clean_and_eval(s):
    s = str(s).strip("[]")
    s = re.sub(r'\s+', ',', s)
    s = re.sub(r',+', ',', s)
    s = s.strip(',')
    s = '[' + s + ']'
    try:
        return ast.literal_eval(s)
    except Exception:
        return [np.nan, np.nan]

def make_bpt(bin_df, ratio_x, err_x, ratio_y, err_y,
             x_label=None, y_label=None,
             title=None, xscale='log', yscale='log',
             xlim=(0.01, 10), ylim=(0.3, 50),
             verbose=True, pickle_filename = None, save_pickle = False):
    
    # Clean error columns
    bin_df[err_x] = bin_df[err_x].apply(clean_and_eval)
    bin_df[err_y] = bin_df[err_y].apply(clean_and_eval)

    initial_count = len(bin_df)

    # Drop rows with missing or malformed values only in key columns
    valid_data = bin_df.dropna(subset=[ratio_x, ratio_y])
    valid_data = valid_data[
        valid_data[err_x].apply(lambda x: isinstance(x, list) and len(x) == 2 and not any(np.isnan(x))) &
        valid_data[err_y].apply(lambda x: isinstance(x, list) and len(x) == 2 and not any(np.isnan(x)))
    ]

    dropped_count = initial_count - len(valid_data)
    if verbose and dropped_count > 0:
        print(f"Dropped {dropped_count} galaxies due to invalid ratios or errors.")

    if len(valid_data) == 0:
        print("No galaxies to plot after filtering.")
        return

    # Extract values and errors
    x = valid_data[ratio_x].to_numpy()
    y = valid_data[ratio_y].to_numpy()
    xerr_array = np.vstack(valid_data[err_x].tolist()).astype(float)
    yerr_array = np.vstack(valid_data[err_y].tolist()).astype(float)
    labels = valid_data['Galaxy ID'].to_numpy()

    # Plot
    fig, ax = plt.subplots(figsize=(8, 7))
    ax.errorbar(x, y, xerr=xerr_array.T, yerr=yerr_array.T,
                fmt='o', color='blue', alpha=0.7)

    # Annotate points
    for xi, yi, label in zip(x, y, labels):
        ax.annotate(label, (xi, yi), fontsize=8, alpha=0.8)

    # Axis labels and styles
    ax.set_xlabel(x_label if x_label else ratio_x, fontsize=14)
    ax.set_ylabel(y_label if y_label else ratio_y, fontsize=14)
    ax.set_title(title if title else f"{ratio_x} vs {ratio_y}", fontsize=15)
    ax.grid(True)
    sns.set_theme(style="ticks")
    fig.tight_layout()

    # Axis scales and limits
    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    print(f"Plotted {len(labels)} galaxies")

    if pickle_filename and save_pickle: 
        with open(pickle_filename, 'wb') as f: 
            pickle.dump(fig, f) 
        if verbose: 
            print(f"Figure saved to {pickle_filename}")
    return fig

In [51]:
proc_glass_df[[col for col in merged_glass_df.columns if col.endswith('err')]]

,CIII]_doublet_flux_err,CIV_doublet_flux_err,OIII]_doublet_flux_err,NeIII_OII_ratio_err,OIII5007_Hbeta_ratio_err,Hdelta_Hbeta_ratio_err,Hgamma_Hbeta_ratio_err
0,"[0.0, 0.0]","[0.022194479326390012, 0.5077333281752245]","[0.24135592238829756, 0.45983210582461026]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
1,"[0.0, 0.0]","[0.35117191393706115, 0.6131278558015487]","[0.012236083237663668, 0.20556971336357965]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
2,"[0.0, 0.0]","[0.1978270755600334, 0.29441677515300435]","[0.2034005466878861, 0.8086768564988434]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
3,NaN,"[0.0, 0.0]","[0.0, 0.0]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
4,"[0.0, 0.0]","[0.33313486415149096, 0.41417928137688154]","[0.46428634230201893, 1.2545622352505992]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
5,"[0.29706767532830824, 0.2699040042134332]","[0.0, 0.008049169321885555]","[0.1553597431665282, 0.20778023856857022]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
6,"[0.5497908898958672, 0.8225215327920723]","[0.0, 0.3307865481666497]","[0.7037839537851529, 0.3386609448996377]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
7,"[0.8330119144296132, 1.5450861714469855]","[36.75535949977759, 184.0487299773702]","[0.3553501352052236, 0.3935942507544262]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
8,"[0.0, 0.15271311644948185]","[0.0, 0.21375944780028577]","[0.08258569266524951, 0.2229044820384653]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
9,"[0.5440579653718205, 0.8762926637323247]","[0.0, 0.0]","[0.0, 0.0]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"


In [24]:
proc_glass_df[[col for col in proc_glass_df.columns if col.endswith('ratio')]]

,NeIII_OII_ratio,OIII5007_Hbeta_ratio,Hdelta_Hbeta_ratio,Hgamma_Hbeta_ratio
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [52]:
make_bpt(merged_glass_df, ratio_x='NeIII_OII_ratio',
    err_x='NeIII_OII_ratio_err',
    ratio_y='OIII5007_Hbeta_ratio',
    err_y='OIII5007_Hbeta_ratio_err')

Dropped 12 galaxies due to invalid ratios or errors.
No galaxies to plot after filtering.
